In [1]:
%load_ext autoreload
%autoreload 2

import unicodecsv as csv
import datetime
import feedparser as fp
import newspaper
import helpers
from newspaper import Source, Article
import multiprocessing 
import pandas as pd

In [2]:
%%time
# Compiling papers with using rss information, producing newspaper objects:
all_cnn_papers = helpers.compile_papers(rss_feeds_dict=helpers.CNN_LINKS_DICT)
all_bbc_papers = helpers.compile_papers(rss_feeds_dict=helpers.BBC_LINKS_DICT)
all_fox_papers = helpers.compile_papers(rss_feeds_dict=helpers.FOX_LINKS_DICT)
all_rt_papers = helpers.compile_papers(rss_feeds_dict=helpers.RT_LINKS_DICT)
# No msnbc rss, so can't used this method with msnbc

[Parse lxml ERR] Unicode strings with encoding declaration are not supported. Please use bytes input or XML fragments without declaration.
[Source parse ERR] http://rss.cnn.com/rss/cnn_topstories.rss
[Parse lxml ERR] Unicode strings with encoding declaration are not supported. Please use bytes input or XML fragments without declaration.
[Source parse ERR] http://rss.cnn.com/rss/cnn_world.rss
[Parse lxml ERR] Unicode strings with encoding declaration are not supported. Please use bytes input or XML fragments without declaration.
[Source parse ERR] http://rss.cnn.com/rss/cnn_us.rss
[Parse lxml ERR] Unicode strings with encoding declaration are not supported. Please use bytes input or XML fragments without declaration.
[Source parse ERR] http://rss.cnn.com/rss/cnn_allpolitics.rss
[Parse lxml ERR] Unicode strings with encoding declaration are not supported. Please use bytes input or XML fragments without declaration.
[Source parse ERR] http://rss.cnn.com/rss/cnn_tech.rss
[Parse lxml ERR] U

In [3]:
%%time
# Download articles, i.e. scrapeoff text and other data:
helpers.art_multithread_downloads(papers=all_cnn_papers, threads_per_source=2)
helpers.art_multithread_downloads(papers=all_bbc_papers, threads_per_source=2)
helpers.art_multithread_downloads(papers=all_fox_papers, threads_per_source=2)
helpers.art_multithread_downloads(papers=all_rt_papers, threads_per_source=2)

CPU times: user 41.2 s, sys: 6.49 s, total: 47.7 s
Wall time: 4min 10s


In [4]:
master_paper_list = [all_cnn_papers, 
                     all_bbc_papers,
                     all_fox_papers,
                     all_rt_papers]

In [5]:
%%time
# Parse all articles:
for paper_list in master_paper_list:
    for paper in paper_list:
        print("Paper:", paper.url)
        paper.parse_articles()

('Paper:', 'http://rss.cnn.com/rss/cnn_topstories.rss')


/Users/alexander/anaconda3/envs/NewsBiasPython2/lib/python2.7/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 24 bytes but only got 0. Skipping tag 2
  " Skipping tag %s" % (size, len(data), tag))
/Users/alexander/anaconda3/envs/NewsBiasPython2/lib/python2.7/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 24 bytes but only got 0. Skipping tag 4
  " Skipping tag %s" % (size, len(data), tag))
/Users/alexander/anaconda3/envs/NewsBiasPython2/lib/python2.7/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 6
  " Skipping tag %s" % (size, len(data), tag))
/Users/alexander/anaconda3/envs/NewsBiasPython2/lib/python2.7/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 24 bytes but only got 0. Skipping tag 7
  " Skipping tag %s" % (size, len(data), tag

/Users/alexander/anaconda3/envs/NewsBiasPython2/lib/python2.7/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 264 bytes but only got 88. Skipping tag 37510
  " Skipping tag %s" % (size, len(data), tag))
/Users/alexander/anaconda3/envs/NewsBiasPython2/lib/python2.7/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 33 bytes but only got 0. Skipping tag 42036
  " Skipping tag %s" % (size, len(data), tag))
/Users/alexander/anaconda3/envs/NewsBiasPython2/lib/python2.7/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 37379
  " Skipping tag %s" % (size, len(data), tag))
/Users/alexander/anaconda3/envs/NewsBiasPython2/lib/python2.7/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 9 bytes but only got 0. Skipping tag 42035
  " Skipping tag %s" % (siz

('Paper:', 'http://rss.cnn.com/rss/cnn_world.rss')


/Users/alexander/anaconda3/envs/NewsBiasPython2/lib/python2.7/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 20 bytes but only got 12. Skipping tag 36867
  " Skipping tag %s" % (size, len(data), tag))
/Users/alexander/anaconda3/envs/NewsBiasPython2/lib/python2.7/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 2060 bytes but only got 788. Skipping tag 59932
  " Skipping tag %s" % (size, len(data), tag))
/Users/alexander/anaconda3/envs/NewsBiasPython2/lib/python2.7/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 2060 bytes but only got 1812. Skipping tag 59932
  " Skipping tag %s" % (size, len(data), tag))
/Users/alexander/anaconda3/envs/NewsBiasPython2/lib/python2.7/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 6928 bytes but only got 188. Skipping tag 37500
  " Skipping 

...

('Paper:', 'http://rss.cnn.com/rss/cnn_health.rss')
('Paper:', 'http://rss.cnn.com/rss/cnn_showbiz.rss')


/Users/alexander/anaconda3/envs/NewsBiasPython2/lib/python2.7/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 17772 bytes but only got 28. Skipping tag 37500
  " Skipping tag %s" % (size, len(data), tag))
/Users/alexander/anaconda3/envs/NewsBiasPython2/lib/python2.7/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 17772 bytes but only got 1052. Skipping tag 37500
  " Skipping tag %s" % (size, len(data), tag))
/Users/alexander/anaconda3/envs/NewsBiasPython2/lib/python2.7/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 17772 bytes but only got 2076. Skipping tag 37500
  " Skipping tag %s" % (size, len(data), tag))
/Users/alexander/anaconda3/envs/NewsBiasPython2/lib/python2.7/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 17772 bytes but only got 3100. Skipping tag 37500
  " S

('Paper:', 'http://rss.cnn.com/rss/cnn_travel.rss')
('Paper:', 'http://rss.cnn.com/rss/cnn_latest.rss')


/Users/alexander/anaconda3/envs/NewsBiasPython2/lib/python2.7/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8218 bytes but only got 208. Skipping tag 37500
  " Skipping tag %s" % (size, len(data), tag))
/Users/alexander/anaconda3/envs/NewsBiasPython2/lib/python2.7/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 50 bytes but only got 0. Skipping tag 42036
  " Skipping tag %s" % (size, len(data), tag))
/Users/alexander/anaconda3/envs/NewsBiasPython2/lib/python2.7/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8218 bytes but only got 1232. Skipping tag 37500
  " Skipping tag %s" % (size, len(data), tag))
/Users/alexander/anaconda3/envs/NewsBiasPython2/lib/python2.7/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8218 bytes but only got 2256. Skipping tag 37500
  " Skipping 

('Paper:', 'http://feeds.bbci.co.uk/news/rss.xml')
('Paper:', 'http://feeds.bbci.co.uk/news/world/rss.xml')
('Paper:', 'http://feeds.bbci.co.uk/news/world/us_and_canada/rss.xml')
('Paper:', 'http://feeds.bbci.co.uk/news/uk/rss.xml')
('Paper:', 'http://feeds.bbci.co.uk/news/business/rss.xml')
('Paper:', 'http://feeds.bbci.co.uk/news/technology/rss.xml')
('Paper:', 'http://feeds.bbci.co.uk/news/science_and_environment/rss.xml')
('Paper:', 'http://feeds.bbci.co.uk/news/entertainment_and_arts/rss.xml')
('Paper:', 'http://feeds.bbci.co.uk/news/health/rss.xml')
('Paper:', 'http://www.feeds.foxnews.com/foxnews/latest')
('Paper:', 'http://feeds.foxnews.com/foxnews/most-popular')
('Paper:', 'http://feeds.foxnews.com/foxnews/entertainment')
('Paper:', 'http://feeds.foxnews.com/foxnews/health')
('Paper:', 'http://feeds.foxnews.com/foxnews/section/lifestyle')
('Paper:', 'http://feeds.foxnews.com/foxnews/politics')
('Paper:', 'http://feeds.foxnews.com/foxnews/science')
('Paper:', 'http://feeds.foxn

In [ ]:
# Run when rt_papers fail in previous cell:
for paper in all_fox_papers:
    print("Paper:", paper.url)
    paper.parse_articles()

In [6]:
def write_out_articles(newspaper_obj, filepath, writeout_type='conventional'):
    """
    TODO: finish this docstring
    :param newspaper_obj:
    :param filepath: str, filepath, WITHOUT the filename
    :param writeout_type: str, 'custom' or 'conventional'
    :return:
    """
    now = datetime.datetime.now()
    current_date = '{}_{}_{}'.format(str(now.year),
                                     str(now.month),
                                     str(now.day))
    if writeout_type == 'custom':
        csv_name = filepath + current_date + '_' + newspaper_obj.description \
                   + '.csv'
    else:
        csv_name = filepath + current_date + '_' + newspaper_obj.brand + '.csv'
    with open(csv_name, 'w') as outcsv:
        writer = csv.writer(outcsv, encoding='utf-8')
        if writeout_type == 'conventional':
            header = ['source_url', 'url', 'title', 'movies', 'text',
                      'keywords', 'meta_keywords', 'tags', 'authors',
                      'publish_date', 'summary', 'html', 'article_html',
                      'meta_description', 'meta_data', 'canonical_link']
        else:
            header = ['source_url', 'url', 'title', 'movies', 'text',
                             'keywords', 'meta_keywords', 'tags', 'authors',
                             'publish_date', 'summary', 'html', 'article_html',
                             'meta_description', 'meta_data', 'canonical_link',
                             'rss_title', 'rss_link', 'rss_id', 'rss_published',
                             'rss_published_parsed', 'rss_feedburner_origlink',
                             'paper_section_name']
        writer.writerow(header)
        for article in newspaper_obj.articles:
            source_url = article.source_url
            url = article.url
            title = article.title
            movies = article.movies
            text = article.text
            keywords = article.keywords
            meta_keywords = article.meta_keywords
            tags = article.tags
            authors = article.authors
            publish_date = article.published_date
            summary = article.summary
            html = article.html
            article_html = article.article_html
            meta_description = article.meta_description
            meta_data = article.meta_data
            canonical_link = article.canonical_link
            if writeout_type == 'custom':
                rss_title = article.additional_data['rss_title']
                rss_link = article.additional_data['rss_link']
                rss_id = article.additional_data['rss_id']
                rss_published = article.additional_data['rss_published']
                rss_published_parsed = \
                    article.additional_data['rss_published_parsed']
                rss_feedburner_origlink = \
                    article.additional_data['rss_feedburner_origlink']
                data_row = [source_url, url, title, movies, text, keywords,
                            meta_keywords, tags, authors, publish_date,
                            summary, html, article_html, meta_description,
                            meta_data, canonical_link, rss_title, rss_link,
                            rss_id, rss_published, rss_published_parsed,
                            rss_feedburner_origlink,
                            newspaper_obj.description]
            else:
                data_row = [source_url, url, title, movies, text, keywords,
                            meta_keywords, tags, authors, publish_date,
                            summary, html, article_html, meta_description,
                            meta_data, canonical_link]
            writer.writerow(data_row)

In [7]:
filepath = 'data_extra/custom_data_pulls/'

In [8]:
%%time
for paper_list in master_paper_list:
    for paper in paper_list:
        write_out_articles(newspaper_obj=paper,
                                   filepath=filepath, 
                                   writeout_type='custom')

CPU times: user 8.18 s, sys: 4.6 s, total: 12.8 s
Wall time: 18.3 s


Check for files:

In [9]:
df = pd.read_csv(filepath+ '2019_3_26_rt_main_page.csv')

In [10]:
df.text

0     Czech tennis player Andrea Hlavackova has ridi...
1     Algeria’s army chief has called for long-term ...
2     The DUP is set to deliver a hammer blow to The...
3     Citigroup has joined JPMorgan and Goldman Sach...
4     A Turkish police officer fired shots at his co...
5     Washington’s sanctions against Moscow have not...
6     After Robert Mueller’s ‘no collusion’ report s...
7     A 42-year-old gay principal who was fired from...
8     NSA whistleblower Edward Snowden has decried t...
9     Agence France-Presse (AFP) employees, at a mee...
10    The Chinese “Belt and Road” initiative is an i...
11    Donald Trump's decision to recognize Israel's ...
12    The European Parliament voted on Tuesday to sc...
13    Yemenis held a mass rally in the capital Sanaa...
14    Russia's renowned figure-skating coach Eteri T...
15    Combat sports retirements are often temporary ...
16    The European Parliament has voted to adopt the...
17    A bid to make a debtor pay a collector fro